

The following code was used to download NO2 data from GEE. Likewise, the methodology for selecting the region of interest and the processing for the generation of daily and monthly time series is presented.


### Earth Engine API and other libraries


In [ ]:
import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=xdTngtksAiR6zE_yM4Bjke_b7PcYVfJHJ85Nhopq1_Q&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWh7oo83Cf0zUZzg_61XDnLVe1fUPfVhanJwOon2nohDXqYUva1CeBg

Successfully saved authorization token.


In [ ]:
import pandas as pd
import numpy as np
import datetime
from dateutil.relativedelta import relativedelta

import matplotlib.pyplot as plt
from google.colab import files
 

import numpy.ma as ma
!pip install geemap
import geemap

!pip install "basemap == 1.3.0b1" "basemap-data == 1.3.0b1" #https://github.com/matplotlib/basemap/issues/525
!pip install basemap

import warnings
warnings.filterwarnings("ignore")


In [ ]:
from mpl_toolkits.basemap import Basemap

## Area of Interest

In [ ]:
ciudad='Buenos Aires'

delta=0.1
if ciudad =='Lima':
   lat_n=-11.5
   lat_s=-12.55
   lon_w=-77.25
   lon_e=-76.60
if ciudad =='San Pablo':
   lat_n=-23.3
   lat_s=-24.05
   lon_w=-46.85
   lon_e=-46.35
if ciudad =='Rio de Janeiro':
   lat_n=-22.72
   lat_s=-23.1
   lon_w=-43.80
   lon_e=-43.08
if ciudad =='Mexico':
   lat_n=19.94
   lat_s=19.08
   lon_w=-99.4
   lon_e=-98.9
if ciudad =='Bogota':
   lat_n=4.85
   lat_s=4.45
   lon_w=-74.25
   lon_e=-74.00
if ciudad =='Santiago':
   lat_n=-33.1
   lat_s=-33.65
   lon_w=-71
   lon_e=-70.15
if ciudad =='Buenos Aires':
   lat_n=-34.52
   lat_s=-34.73
   lon_w=-58.56
   lon_e=-58.33 
   delta = 0.2

rdp = ee.Geometry.Rectangle([np.round(lon_w-delta,2), np.round(lat_s-delta,2), np.round(lon_e+delta,2), np.round(lat_n+delta,2)],geodesic= False,proj='EPSG:4326')



## Tiff

In [ ]:
##para exportar tiff

start = datetime.datetime.strptime("2019-04-01", "%Y-%m-%d")
end = datetime.datetime.strptime("2020-05-01", "%Y-%m-%d")
r = relativedelta(end, start)
meses=r.years*12+r.months
date_generated = [start + relativedelta(months=x) for x in range(0, meses)]

for ii, date in enumerate(date_generated):
      inicio=date.strftime("%Y-%m-%d")
      ini=date.strftime("%Y_%m")
      fin=date + relativedelta(days=1)
      final=fin.strftime("%Y-%m-%d")

      collection=ee.ImageCollection('COPERNICUS/S5P/OFFL/L3_NO2').select('tropospheric_NO2_column_number_density').filterDate(inicio,final)
      collection_img_mean=collection.mean().setDefaultProjection(collection.first().projection())
      geemap.ee_export_image_to_drive(collection_img_mean, description=ciudad+'_' + ini, region=rdp, scale=1113.2)


## Masks

In [ ]:
qs=[0.7]  ##pueden cambiar el percentil que se quiera
eq=[]
start = datetime.datetime.strptime("2018-07-01", "%Y-%m-%d")
end = datetime.datetime.strptime("2020-03-01", "%Y-%m-%d")
r = relativedelta(end, start)
meses=r.years*12+r.months
date_generated = [start + relativedelta(months=x) for x in range(0, meses)]
for q in qs:  
  e=[]
  for ii, date in enumerate(date_generated):
        inicio=date.strftime("%Y-%m-%d")
        ini=date.strftime("%Y_%m")
        fin=date + relativedelta(months=1)
        final=fin.strftime("%Y-%m-%d")
        print('media entre', inicio, 'y',final)

        if ciudad == 'Bogota': ##la mascara se hace en función del NO2 total, por el problema de los nans.
           collection=ee.ImageCollection('COPERNICUS/S5P/OFFL/L3_NO2').select('NO2_column_number_density').filterDate(inicio,final)
        else:
           collection=ee.ImageCollection('COPERNICUS/S5P/NRTI/L3_NO2').select('tropospheric_NO2_column_number_density').filterDate(inicio,final)
        collection_img=collection.mean().setDefaultProjection(collection.first().projection())
        ax=geemap.ee_to_numpy(collection_img,region=rdp)
        

        if ax is None:
          pass
        else: 
            axx=ax[:,:,0]
            no2q=np.quantile(axx,q)
            b=ma.masked_less_equal(axx, no2q)
            unos=np.ones(axx.shape)
            unos_=ma.array(unos, mask=b.mask)

            e.append(unos_)

  eee=sum(e)
  eee_val=np.max(eee)
  eee=sum(e)/eee_val

  am = ma.MaskedArray(eee,fill_value=0)
  amm=am.filled()
  amm=amm.astype(int)
  eq.append(amm)

  #print(np.sum(amm)) ## número de pixeles de la mascara


  aa, bb = amm.shape
  fig=plt.figure(figsize = (8,8))

  lats=np.round(np.linspace(np.round(lat_s-delta,2),np.round(np.round(lat_s-delta,2)+(aa-1)*0.01,2),aa),2)
  lons=np.round(np.linspace(np.round(lon_w-delta,2),np.round(np.round(lon_w-delta,2)+(bb-1)*0.01,2),bb),2)
  
  X,Y=np.meshgrid(lons,lats)

  m = Basemap(projection='merc',llcrnrlon=lon_w-delta,llcrnrlat=lat_s-delta,urcrnrlon=lon_e+delta-0.01,urcrnrlat=lat_n+delta-0.01,resolution='i')
  
  x, y = m(X, Y)
  m.drawcoastlines()
  m.drawcountries()
  m.drawlsmask(land_color='white', ocean_color='white') 
  #m.readshapefile(mapa,'partidos',drawbounds=True) 
  parallels = np.arange(lats[0],lats[-1],(lats[-1]-lats[0])/5)  
  meridians = np.arange(lons[0],lons[-1],(lons[-1]-lons[0])/5)  
  m.drawparallels(parallels,labels=[1,0,0,0],fontsize=10)
  m.drawmeridians(meridians,labels=[0,0,0,1],fontsize=10)
  iny=plt.pcolormesh(x,y,amm,vmin=0,vmax=1, cmap='jet')
  CS= plt.contour(x,y,amm,levels=[0.0])
  ##Cuando se realizan los contour lines, se generan varias curvas, se selecciona la que coincide con la región urbana. Se grafica y se guarda
  if ciudad == 'Bogota':
    dat0= CS.allsegs[0][0]
    dat1= CS.allsegs[0][1]
    dat01=np.concatenate((dat0,dat1))
    final=dat01[0].reshape(1,-1)
    dao=np.concatenate((dat01,final))
    plt.plot(dao[:,0],dao[:,1],color='white')
    plt.savefig('maskara_'+ciudad+str(int(q*100))+'.png')
    plt.close()
    files.download('maskara_'+ciudad+str(int(q*100))+'.png') 

    #Se guardan las coordenadas de la máscara principal  
    CS= plt.contour(X,Y,amm,levels=[0.0])
    dat0= CS.allsegs[0][0]
    dat1= CS.allsegs[0][1]
    dat01=np.concatenate((dat0,dat1))
    final=dat01[0].reshape(1,-1)
    dao=np.concatenate((dat01,final))
    dao=np.round(dao,2)

    dao_dic={'Lon':dao[:,0],'Lat':dao[:,1]}
    dao_df=pd.DataFrame(data=dao_dic)
    dao_df.to_csv('coordMask_'+ciudad+'.csv',index=False)
    files.download('coordMask_'+ciudad+'.csv')
  
  else:
    if ciudad in ['Buenos Aires', 'Lima','Rio de Janeiro','Bogota']:
      px=0
    if ciudad in ['Mexico', 'San Pablo']:
      px=1
    
    dat0= CS.allsegs[0][px]
    plt.plot(dat0[:,0],dat0[:,1],color='white')
    plt.savefig('maskara_'+ciudad+str(int(q*100))+'.png')
    plt.close()
    files.download('maskara_'+ciudad+str(int(q*100))+'.png') 

    
    #Se guardan las coordenadas de la máscara principal  
    CS= plt.contour(X,Y,amm,levels=[0.0])
    dat0= CS.allsegs[0][px]
    dat0=np.round(dat0,2)
    dat0_dic={'Lon':dat0[:,0],'Lat':dat0[:,1]}
    dat0_df=pd.DataFrame(data=dat0_dic)
    dat0_df.to_csv('coordMask_'+ciudad+'.csv',index=False)
    files.download('coordMask_'+ciudad+'.csv')



## Region reduction function

Functions from https://developers.google.com/earth-engine/tutorials/community/time-series-visualization-with-altair?hl=sr


In [ ]:
def create_reduce_region_function(geometry,
                                  reducer=ee.Reducer.mean(),
                                  scale=1000,
                                  crs='EPSG:4326',
                                  bestEffort=True,
                                  maxPixels=1e13,
                                  tileScale=4):

  def reduce_region_function(img):

    stat = img.reduceRegion(
        reducer=reducer,
        geometry=geometry,
        scale=scale,
        crs=crs,
        bestEffort=bestEffort,
        maxPixels=maxPixels,
        tileScale=tileScale)

    return ee.Feature(geometry, stat).set({'millis': img.date().millis()})
  return reduce_region_function

# Define a function to transfer feature properties to a dictionary.
def fc_to_dict(fc):
  prop_names = fc.first().propertyNames()
  prop_lists = fc.reduceColumns(
      reducer=ee.Reducer.toList().repeat(prop_names.size()),
      selectors=prop_names).get('list')

  return ee.Dictionary.fromLists(prop_names, prop_lists)

# Function to add date variables to DataFrame.
def add_date_info(df):
  df['Timestamp'] = pd.to_datetime(df['millis'], unit='ms')
  df['Year'] = pd.DatetimeIndex(df['Timestamp']).year
  df['Month'] = pd.DatetimeIndex(df['Timestamp']).month
  df['Day'] = pd.DatetimeIndex(df['Timestamp']).day
  df['DOY'] = pd.DatetimeIndex(df['Timestamp']).dayofyear
  return df

## Download of raw time series using previous masks

In [ ]:
ciudad='Buenos Aires'

### archivos de las coordenadas 
if ciudad == 'Buenos Aires':
  coord_file ='https://drive.google.com/file/d/1y_liYcgc5FpWzZR-ykFJfLOxZaw-IQgk/view?usp=sharing'
if ciudad == 'Bogota':
  coord_file ='https://drive.google.com/file/d/1b4L01O16QVQ8O81y8mdojbS-N179bf6I/view?usp=sharing'
if ciudad == 'San Pablo':
  coord_file ='https://drive.google.com/file/d/1HDuap8AEhGHvrEtxDCXQSqVYOSuGZYZn/view?usp=sharing'
if ciudad == 'Rio de Janeiro':
  coord_file ='https://drive.google.com/file/d/1ZMjDyhsonpGMD2H_QpUp-VWANeZV8KNv/view?usp=sharing'
if ciudad == 'Lima':
  coord_file ='https://drive.google.com/file/d/1mM9h-RcPcO3m_hy0GH5_JDutAfh8BUyd/view?usp=sharing'
if ciudad == 'Mexico':
  coord_file ='https://drive.google.com/file/d/1ON0Fl7YIp8LBWjxrRXyHRmSABmt6Zz06/view?usp=sharing'
  
url=coord_file
url2='https://drive.google.com/uc?id=' + url.split('/')[-2]
coord_df=pd.read_csv(url2)

coord=np.zeros((len(coord_df.Lon.values),2))
coord[:,0]=coord_df.Lon.values
coord[:,1]=coord_df.Lat.values
lista_coord=coord.tolist()
#print(lista_coord)

rdp = ee.Geometry.Polygon(lista_coord)
inicio=['2018-07-01','2019-01-01','2020-01-01']
final =['2019-01-01','2020-01-01','2020-12-01']
    
reduce_mean = create_reduce_region_function(geometry=rdp, reducer=ee.Reducer.mean(), scale=1113.2, crs='EPSG:4326')
reduce_std = create_reduce_region_function(geometry=rdp, reducer=ee.Reducer.sampleStdDev(), scale=1113.2, crs='EPSG:4326')
for i in range(3):

    year=inicio[i][:4]

    gas_trop=ee.ImageCollection('COPERNICUS/S5P/OFFL/L3_NO2').select('tropospheric_NO2_column_number_density').filterDate(inicio[i],final[i])
    gas_trop_mean_fc = ee.FeatureCollection(gas_trop.map(reduce_mean)).filter(
        ee.Filter.notNull(gas_trop.first().bandNames()))
    gas_trop_std_fc = ee.FeatureCollection(gas_trop.map(reduce_std)).filter(
        ee.Filter.notNull(gas_trop.first().bandNames()))
 
    gas_trop_mean_dict=fc_to_dict(gas_trop_mean_fc).getInfo()
    gas_trop_std_dict=fc_to_dict(gas_trop_std_fc).getInfo()
 
    gas_trop_mean_df=pd.DataFrame(gas_trop_mean_dict)
    gas_trop_std_df=pd.DataFrame(gas_trop_std_dict)

    gas_trop_mean_df =  add_date_info(gas_trop_mean_df)
    gas_trop_mean_df  = gas_trop_mean_df.rename(columns={'tropospheric_NO2_column_number_density': 'NO2_trop_mean'}).drop(columns=['millis', 'system:index'])
    gas_trop_mean_df['NO2_trop_std']=gas_trop_std_df['tropospheric_NO2_column_number_density']

    gas_trop_mean_df.to_csv('NO2_trop_diario'+str(year)+'_'+ciudad+'.csv',index=True)
    files.download('NO2_trop_diario'+str(year)+'_'+ciudad+'.csv')

## Daily and monthly time series preprocessing code


In [ ]:
import datetime
megalopolis=['Buenos Aires','Bogota','Lima','Mexico','San Pablo','Rio de Janeiro']
pixeles=[259,367,1986,1165,633,750]

gas='NO2_trop'
gasname='NO2 troposférico'

for indice, ciudad in enumerate(megalopolis):
    
    if ciudad == 'Buenos Aires':
      url2018 ='https://drive.google.com/file/d/1aUUrPR41L4jYB8QfhCWOqUNxQwzJNU9G/view?usp=sharing'
      url2019 ='https://drive.google.com/file/d/1SVoyHZ0eyFHUTVUs1beKtFx9xPkntk_3/view?usp=sharing'
      url2020 ='https://drive.google.com/file/d/1JaHzmnsjwCMNHr3NL5oYeBrcHobNaw7b/view?usp=sharing'
    if ciudad == 'Bogota':
      url2018 ='https://drive.google.com/file/d/1I-DtOUmnCO06eZlsLDZjekdnnmnoKvoR/view?usp=sharing'
      url2019 ='https://drive.google.com/file/d/1mTmWOeJP5xZr7THt1HlYAWJ9Q5CcIfTh/view?usp=sharing'
      url2020 ='https://drive.google.com/file/d/1YKGZhcFFWW1vtNo7JGbxGqmZLf16KSQi/view?usp=sharing'
    if ciudad == 'Lima':
      url2018 ='https://drive.google.com/file/d/1LF6mRtXFAL4lSWyflSroKW3miVM4ekjz/view?usp=sharing'
      url2019 ='https://drive.google.com/file/d/116w1GbaZ2CMknQmfQ-7oDohzU7650Kkk/view?usp=sharing'
      url2020 ='https://drive.google.com/file/d/1rZt9F5m7dA94HrDLre1CADCJftm2KmYa/view?usp=sharing'
    if ciudad == 'San Pablo':
      url2018 ='https://drive.google.com/file/d/1zIKs2fyKFm8TLfBjssgXr1u9Mpd8Y2bM/view?usp=sharing'
      url2019 ='https://drive.google.com/file/d/1oFl8mDW9twrjKWw38KBJT_jZYp3bCfWm/view?usp=sharing'
      url2020 ='https://drive.google.com/file/d/1AU4N2uRSuw8qXMBxqRe4mH8K9l6d_07-/view?usp=sharing'
    if ciudad == 'Rio de Janeiro':
      url2018 ='https://drive.google.com/file/d/1HW3eLUf-ZTfspCk_GEbr-RsOSuPFZQhv/view?usp=sharing'
      url2019 ='https://drive.google.com/file/d/1hfXgfXXBeJwDIp1Si5i2z_p_ZKWFQ__o/view?usp=sharing'
      url2020 ='https://drive.google.com/file/d/18UnYiYLhGgZff9uT23oqWsmUbz_0cThW/view?usp=sharing'
    if ciudad == 'Mexico':
      url2018 ='https://drive.google.com/file/d/1RDHn15H2n-JJiZCQl82dk1fjIP6ZF6uo/view?usp=sharing'
      url2019 ='https://drive.google.com/file/d/1bRbK1tc8vLLMpFQeZXY1wsqBmdO_AHv4/view?usp=sharing'
      url2020 ='https://drive.google.com/file/d/1al0jmtdZ6EeJva9xMrwSvBlYet26KI6Y/view?usp=sharing'                        

    url2018_2='https://drive.google.com/uc?id=' + url2018.split('/')[-2]
    url2019_2='https://drive.google.com/uc?id=' + url2019.split('/')[-2]
    url2020_2='https://drive.google.com/uc?id=' + url2020.split('/')[-2]

    df1=pd.read_csv(url2018_2)
    df2=pd.read_csv(url2019_2)
    df3=pd.read_csv(url2020_2)
    
    df3=df3.drop(df3[df3.DOY > 333].index)  ### saco los valores posteriores a 27 de nov de 2020 por calibracion
    df=df1.append([df2,df3],ignore_index=True)
    df_diario=df.groupby(['Year','Month','Day']).mean()
    
    df_gas=df_diario.reset_index()
    df_gas['Fecha_datetime']=pd.to_datetime(df_gas['Year'].astype(str)+'-'+df_gas['Month'].astype(str)+'-'+df_gas['Day'].astype(str),format='%Y-%m-%d')
    #df_gas['dias']=df_gas['Fecha_datetime'].map(datetime.datetime.toordinal)
    #df_gas['DOW']=df_gas['Fecha_datetime'].dt.dayofweek
    #df_gas['dias']=df_gas['dias']-736876

    df_gas.drop(columns=['DOY','Unnamed: 0'],inplace=True)
    columna=gas+'_mean' 
  
    #df_gas= df_gas.drop(columns=['dias','DOW'])
    df_gas['var_e']=df_gas['NO2_trop_std']**2  ##error de la media epsacial al cuadrado
    df_mensual=df_gas.groupby(['Year','Month']).mean().reset_index().drop(columns=['Day','NO2_trop_std'])
    df_mensual['N_days']=df_gas.groupby(['Year', 'Month'])['NO2_trop_mean'].describe().reset_index()['count']
    df_mensual['SEM']=(df_mensual['var_e']/df_mensual['N_days'])**.5
    df_mensual=df_mensual.drop(columns=['var_e'])
    df_mensual['Fecha_datetime']=pd.to_datetime(df_mensual['Year'].astype(str)+'-'+df_mensual['Month'].astype(str),format='%Y-%m')

    df_gas['SEM']=df_gas['NO2_trop_std']/np.sqrt(pixeles[indice])
    df_gas.drop(columns=['Fecha_datetime','var_e'],inplace=True)

    df_gas.to_csv(ciudad+'NO2_trop_diario.csv',index=False)
    files.download(ciudad+'NO2_trop_diario.csv')
    df_mensual.to_csv(ciudad+'NO2_trop_mensual.csv',index=False)
    files.download(ciudad+'NO2_trop_mensual.csv')

    indice_2019=np.logical_and(df_mensual['Year']==2019,df_mensual['Month']==4)
    indice_2020=np.logical_and(df_mensual['Year']==2020,df_mensual['Month']==4)
    print('La caída interanual en',ciudad,'durante abril es del:',np.round(100*(
        df_mensual[indice_2020]['NO2_trop_mean'].values[0]-df_mensual[indice_2019]['NO2_trop_mean']).values[0]/
                                                                       df_mensual[indice_2019]['NO2_trop_mean'].values[0],2),'%')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

La caída interanual en Buenos Aires durante abril es del: -56.25 %


...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

La caída interanual en Rio de Janeiro durante abril es del: -39.08 %
